In [82]:
# imports
import pandas as pd
import requests
import pickle as p
import matplotlib.pyplot as plt

In [35]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [39]:
census = pd.read_pickle('./Cap-Census-data/DP03_clean.p')
census.head().T[:10]

,1,2,3,4,5
0,,,,,
Geographic Area Name,"Census Tract 626.44, Orange County, California","Census Tract 626.40, Orange County, California","Census Tract 630.08, Orange County, California","Census Tract 631.03, Orange County, California","Census Tract 637.01, Orange County, California"
GEO_ID,1400000US06059062644,1400000US06059062640,1400000US06059063008,1400000US06059063103,1400000US06059063701
Estimate EMPLOYMENT STATUS Population 16 years and over,6767,3062,833,2189,4348
Estimate EMPLOYMENT STATUS Population 16 years and over In labor force,3967,2446,431,1514,3106
Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force,3967,2436,431,1514,3106
Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed,3811,2320,412,1407,2937
Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed,156,116,19,107,169
Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces,0,10,0,0,0
Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force,2800,616,402,675,1242


In [7]:
# census.columns.tolist()

['Geographic Area Name',
 'GEO_ID',
 'Estimate EMPLOYMENT STATUS Population 16 years and over',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Employed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Civilian labor force Unemployed',
 'Estimate EMPLOYMENT STATUS Population 16 years and over In labor force Armed Forces',
 'Estimate EMPLOYMENT STATUS Population 16 years and over Not in labor force',
 'Estimate EMPLOYMENT STATUS Civilian labor force',
 'Estimate EMPLOYMENT STATUS Civilian labor force Unemployment Rate',
 'Estimate EMPLOYMENT STATUS Females 16 years and over',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force',
 'Estimate EMPLOYMENT STATUS Females 16 years and over In labor force Civilian labor force',
 'Estimate

In [40]:
census.shape

(583, 143)

In [17]:
583*3      # if we have 583 tracts, and estimate 3 block groups per tract, that would calculate 1749 block groups.  Voting data implies ~1650 precincts.

1749

### Understand Vote results files and what demographics are included

In [69]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

In [68]:
vote = pd.read_csv('./OC-c059_g14_sov_data_by_g14_srprec.csv')
vote.shape
# 75 variables, 1700 rows

(1736, 75)

In [70]:
vote.head()

,county,srprec,addist,cddist,sddist,...,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01
0,30,10316,65,47,29,...,260,180,202,194,230
1,30,10317,65,47,29,...,136,103,117,135,124
2,30,10319,65,47,29,...,180,99,135,128,153
3,30,10320,65,47,29,...,311,197,222,231,266
4,30,10322,65,47,29,...,216,169,173,181,192


In [72]:
vote.isna().sum()
#good news - no nulls

county      0
srprec      0
addist      0
cddist      0
sddist      0
           ..
SOSREP01    0
SPINOP01    0
SPINOP02    0
TRSDEM01    0
TRSREP01    0
Length: 75, dtype: int64

In [101]:
vote.info()
# most columns are integers, except for SRPREC, which is a categorical label.  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1736 entries, 0 to 1735
Data columns (total 75 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   county    1736 non-null   int64 
 1   srprec    1736 non-null   object
 2   addist    1736 non-null   int64 
 3   cddist    1736 non-null   int64 
 4   sddist    1736 non-null   int64 
 5   bedist    1736 non-null   int64 
 6   TOTREG    1736 non-null   int64 
 7   DEMREG    1736 non-null   int64 
 8   REPREG    1736 non-null   int64 
 9   AIPREG    1736 non-null   int64 
 10  GRNREG    1736 non-null   int64 
 11  LIBREG    1736 non-null   int64 
 12  NLPREG    1736 non-null   int64 
 13  REFREG    1736 non-null   int64 
 14  DCLREG    1736 non-null   int64 
 15  MSCREG    1736 non-null   int64 
 16  TOTVOTE   1736 non-null   int64 
 17  DEMVOTE   1736 non-null   int64 
 18  REPVOTE   1736 non-null   int64 
 19  AIPVOTE   1736 non-null   int64 
 20  GRNVOTE   1736 non-null   int64 
 21  LIBVOTE   1736

In [98]:
reg = pd.read_csv('./OC-c059_g14_voters_by_g14_srprec.csv')
reg.shape
# 140 variables, 1661 rows

(1661, 140)

In [121]:
reg.head()

,election,type,srprec,totreg_r,dem,...,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g
0,g14,V,10316,454,141,...,1,1,1,0,2
1,g14,V,10317,268,116,...,1,3,0,0,1
2,g14,V,10319,297,93,...,0,0,0,0,2
3,g14,V,10320,502,141,...,0,2,0,0,4
4,g14,V,10322,384,140,...,1,1,1,0,1


In [109]:
reg.info()
#why is it 1.8 MEGAbytes?  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661 entries, 0 to 1660
Columns: 140 entries, election to oreg9g
dtypes: int64(138), object(2)
memory usage: 1.8+ MB


In [104]:
(reg['srprec']).dtype

dtype('int64')

In [119]:
reg_cols = reg.columns.tolist()
[print((reg[col]).dtype) for col in reg_cols]
#first two columns are objects:  ['election'] and ['type'] ... they are categories

object
object
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [120]:
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661 entries, 0 to 1660
Columns: 140 entries, election to oreg9g
dtypes: int64(138), object(2)
memory usage: 1.8+ MB


In [126]:
reg.isna().sum().sort_values()
#confirmed no nulls

election    0
othf5564    0
othf4554    0
othf3544    0
othf2534    0
           ..
filoth      0
demmunk     0
demm1824    0
vietoth     0
oreg9g      0
Length: 140, dtype: int64

In [95]:
#why are SOV and SOR different lengths/have different # of columns?  

vote['srprec'].sort_values()

0         10316
1         10317
2         10319
3         10320
4         10322
         ...   
1731      75112
1732      75114
1733      75115
1734      75601
1735    CNTYTOT
Name: srprec, Length: 1736, dtype: object

In [80]:
reg['srprec'].sort_values()

247      2001
248      2002
249      2003
250      2007
251      2008
        ...  
1656    75111
1657    75112
1658    75114
1659    75115
1660    75601
Name: srprec, Length: 1661, dtype: int64

In [92]:
reg.loc[reg['srprec'] == 10316]

,election,type,srprec,totreg_r,dem,...,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g
0,g14,V,10316,454,141,...,1,1,1,0,2


In [96]:
reg.loc[vote['srprec'] == '10316']

,election,type,srprec,totreg_r,dem,...,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g
0,g14,V,10316,454,141,...,1,1,1,0,2


In [128]:
len(vote) - len(reg)
# vote df has 75 more columns than reg.  

75

In [142]:
srprec = pd.DataFrame(vote['srprec'])

In [143]:
srprec.rename(columns={'srprec': 'vote_srprec'})

,vote_srprec
0,10316
1,10317
2,10319
3,10320
4,10322
...,...
1731,75112
1732,75114
1733,75115
1734,75601


In [144]:
srprec['reg_srprec'] = reg['srprec']
srprec.tail()

,srprec,reg_srprec
1731,75112,NaN
1732,75114,NaN
1733,75115,NaN
1734,75601,NaN
1735,CNTYTOT,NaN


In [148]:
srprec.loc[srprec['reg_srprec'].isna() == True]

,srprec,reg_srprec
1661,71362,NaN
1662,71368,NaN
1663,71378,NaN
1664,7138,NaN
1665,7139,NaN
...,...,...
1731,75112,NaN
1732,75114,NaN
1733,75115,NaN
1734,75601,NaN


In [164]:
# df.drop(df.index[df['line_race'] == 0], inplace = True)
srprec.drop(srprec.index[srprec['srprec'] == 'CNTYTOT'], inplace=True)

In [165]:
srprec.iloc[1660]

srprec        71361
reg_srprec    75601
Name: 1660, dtype: object

In [171]:
srprec.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1735 entries, 0 to 1734
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   srprec      1735 non-null   float64
 1   reg_srprec  1661 non-null   float64
dtypes: float64(2)
memory usage: 40.7 KB


In [170]:
srprec['srprec'] = srprec['srprec'].astype(float)

In [176]:
# df[df['Col2'].isnull()]
srprec[srprec['srprec'].isna()]

,srprec,reg_srprec


### Mapping scratch pad:

In [220]:
#stack
# !pip install simpledbf
import simpledbf
from simpledbf import Dbf5

In [221]:
#look at geo files from SWDB and see what's in there:
dbf = Dbf5('2011_congressional_state_equiv.dbf')
df = dbf.to_dataframe()

# ?? uses census reference id's?  not geographic coordinates.

In [222]:
df.head()

,BLOCK,DISTRICTID
0,060650432722072,42
1,060650432722026,42
2,060650432722070,42
3,060650432722071,42
4,060650432722025,42


In [223]:
df.shape

(710144, 2)

In [224]:
x = df['DISTRICTID'].value_counts()
len(x)

53

In [225]:
df['BLOCK'].value_counts()[:10]

060190016002023    1
060290033031342    1
060710023051027    1
060710103004225    1
060650469001763    1
060990018002073    1
060670043004021    1
060710092021033    1
061110002001176    1
060990006021037    1
Name: BLOCK, dtype: int64

In [226]:
df.groupby('DISTRICTID').count().tail()
#53 congressional districts in 2011

,BLOCK
DISTRICTID,
49,9993
50,13137
51,15763
52,8717
53,7899


In [227]:
x = df.loc[df['DISTRICTID'] == '53']
x[:10]

,BLOCK,DISTRICTID
10016,060730001001016,53
10017,060730001001006,53
10018,060730001001000,53
10019,060730001001017,53
10020,060730001001018,53
10021,060730001001030,53
10022,060730001001008,53
10023,060730001001004,53
10024,060730001001001,53
10025,060730001001003,53


In [228]:
x.shape   #7899 blocks in Dist53

(7899, 2)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710144 entries, 0 to 710143
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   BLOCK       710144 non-null  object
 1   DISTRICTID  710144 non-null  object
dtypes: object(2)
memory usage: 10.8+ MB


In [230]:
# pandas documentation
# s.str.slice(start=0, stop=5, step=3)
# s.stri.slice[0:5:3]
df['state'] = df['BLOCK'].str.slice(stop=2)
df['county'] = df['BLOCK'].str.slice(start=2, stop=5)
df['tract'] = df['BLOCK'].str.slice(start=5, stop=11)
df['block'] = df['BLOCK'].str.slice(start=11, stop=15)

In [232]:
w = df.loc[df['DISTRICTID'] == '53']
w[:10]
# df.drop(columns=['tracts'])

,BLOCK,DISTRICTID,state,county,tract,block
10016,060730001001016,53,06,073,000100,1016
10017,060730001001006,53,06,073,000100,1006
10018,060730001001000,53,06,073,000100,1000
10019,060730001001017,53,06,073,000100,1017
10020,060730001001018,53,06,073,000100,1018
10021,060730001001030,53,06,073,000100,1030
10022,060730001001008,53,06,073,000100,1008
10023,060730001001004,53,06,073,000100,1004
10024,060730001001001,53,06,073,000100,1001
10025,060730001001003,53,06,073,000100,1003


In [233]:
df[:10]

,BLOCK,DISTRICTID,state,county,tract,block
0,060650432722072,42,06,065,043272,2072
1,060650432722026,42,06,065,043272,2026
2,060650432722070,42,06,065,043272,2070
3,060650432722071,42,06,065,043272,2071
4,060650432722025,42,06,065,043272,2025
5,060650432722027,42,06,065,043272,2027
6,060650464011053,42,06,065,046401,1053
7,060650432722017,42,06,065,043272,2017
8,060650464011052,42,06,065,046401,1052
9,060650464011058,42,06,065,046401,1058


In [236]:
df.loc[df['county'] == '065'].nunique()

BLOCK         35718
DISTRICTID        4
state             1
county            1
tract           453
block          2003
dtype: int64

### Srprec <> Census block/tract -- explore

In [241]:
xlate = pd.read_csv('./map_files/c059_g12_sr_blk_map.csv')
xlate.shape

(23490, 8)

In [250]:
xlate.tail()

,srprec,tract,block,blkreg,srtotreg,pctsrprec,blktotreg,pctblk
23485,75602,75809,2007,604,1437,42.032011,613,98.531811
23486,75602,75814,2003,14,1437,0.974252,14,100.000000
23487,75602,75814,2016,2,1437,0.139179,2,100.000000
23488,75602,75814,2017,28,1437,1.948504,28,100.000000
23489,75901,75809,2007,9,9,100.000000,613,1.468189


In [244]:
xlate['tract'].nunique()
#582 census tracts in g12 data set.

582

In [245]:
xlate['srprec'].nunique()
#1877 srprec's

1877

In [246]:
xlate['block'].nunique()

558

In [247]:
xlate['tract'].value_counts()

63500    235
62800    178
62702    141
52420    138
62604    132
        ... 
63008      6
99502      6
21813      5
52518      4
52404      2
Name: tract, Length: 582, dtype: int64

In [248]:
xlate['block'].value_counts()

2001    468
2002    461
2003    449
2004    434
2000    423
       ... 
7015      1
9008      1
2165      1
2133      1
6053      1
Name: block, Length: 558, dtype: int64

In [249]:
xlate['srprec'].value_counts()

2193     74
53355    58
68296    57
13377    56
68098    55
         ..
2434      1
69902     1
5379      1
44123     1
56383     1
Name: srprec, Length: 1877, dtype: int64